<a href="https://colab.research.google.com/github/codedm24/Transformers/blob/Transformers-Intro/Multiple-Sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import transformers

In [ ]:
!pip install transformers[sentencepiece]

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "What does machine learning do?"

tokens = tokenizer.tokenize(sequence)
print(f"tokens: {tokens}")
ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"ids: {ids}")

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokens: ['what', 'does', 'machine', 'learning', 'do', '?']
ids: [2054, 2515, 3698, 4083, 2079, 1029]
Input IDs: tensor([[2054, 2515, 3698, 4083, 2079, 1029]])
Logits: tensor([[ 3.5672, -2.8648]], grad_fn=<AddmmBackward0>)


Padding sequence ids

In [ ]:
sequence1_ids = [[200,200,200]]
sequence2_ids = [[200,200]]
batched_ids = [
    [200,200,200],
    [200,200,tokenizer.pad_token_id]
]


Padding the inputs

In [2]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequence_ids1 = [[200,200,200]]
sequence_ids2 = [[200,200]]
batched_ids = [
    [200,200,200],
    [200,200,tokenizer.pad_token_id]
]
print(model(torch.tensor(sequence_ids1)))
print(model(torch.tensor(sequence_ids2)))
print(model(torch.tensor(batched_ids)))

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


Padding the inputs with attention mask

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequence_ids1 = [[200,200,200]]
sequence_ids2 = [[200,200]]
batched_ids = [
    [200,200,200],
    [200,200,tokenizer.pad_token_id]
]

attention_mask = [
    [1,1,1],
    [1,1,0]
]

print(torch.tensor(sequence_ids1))
print(torch.tensor(sequence_ids2))
print(torch.tensor(batched_ids))
print(torch.tensor(attention_mask))

print(model(torch.tensor(sequence_ids1)))
print(model(torch.tensor(sequence_ids2)))
print(model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask)))


#print(tokenizer.decode(torch.tensor(sequence_ids1)))
#print(tokenizer.decode(sequence_ids1["input_ids"]))

tensor([[200, 200, 200]])
tensor([[200, 200]])
tensor([[200, 200, 200],
        [200, 200,   0]])
tensor([[1, 1, 1],
        [1, 1, 0]])
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


Using Tokenizer

In [13]:
sequence = ["What does Machine Learning do?"]
model_inputs = tokenizer(sequence)
print(f"model inputs: {model_inputs}")
print(tokenizer.decode(torch.tensor(sequence_ids1)))

model inputs: {'input_ids': [[101, 2054, 2515, 3698, 4083, 2079, 1029, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}


TypeError: argument 'ids': 'dict' object cannot be converted to 'Sequence'